In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:23, 2406.13it/s]


In [4]:
responses[0]['text']

'Добрый день! Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.  Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года. Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован. Каким образом Вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор, мне по меньшей мере не понятно,\xa0 и почему по Вашей милости я должна переживать и бояться за себе и свое имущество. Письма доставляют мн

In [5]:
from rusenttokenize import ru_sent_tokenize

In [6]:
sents = []
for resp in tqdm(responses):
    token = ru_sent_tokenize(resp['text'])
    for t in token:
        sents.append(t)

100%|████████████████████████████████| 153499/153499 [01:39<00:00, 1538.78it/s]


In [7]:
len(sents)

2503274

In [8]:
from pymorphy2 import MorphAnalyzer
pm2 = MorphAnalyzer()

In [9]:
%%time
lemmas_sents = []
for l in sents[:100000]:
    lemm = [pm2.parse(word)[0].normal_form for word in l.split()]
    lemmas_sents.append(' '.join(lemm))

Wall time: 4min 37s


In [10]:
len(lemmas_sents)

100000

In [11]:
from nltk.corpus import stopwords

mystopwords = stopwords.words('russian') + ['c','это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

mystoplemmas = ['который','прошлый','сей', 'свой', 'наш', 'мочь']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return [token for token in text.split() if not token in mystoplemmas]
    except:
        return ""
def remove_all(text):
    t1 = remove_stopwords(text)
    return remove_stoplemmas(t1)

In [12]:
%%time
sw = lambda x: remove_all(x)
wo_stop = list(map(sw, lemmas_sents))

Wall time: 2.37 s


In [13]:
wo_stop[1]

['являться',
 'клиент',
 'банка',
 'поручитель',
 'кредитный',
 'договору,',
 'являться',
 'какой-либо',
 'лицом,',
 'письменно',
 'оформить',
 'отношение',
 'банк',
 'повод',
 'урегулирование',
 'чей-либо',
 'задолженности.']

In [14]:
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [15]:
%%time

model = Word2Vec(wo_stop, size=100, window=3, min_count=20, workers=7)

Wall time: 2.13 s


In [16]:
model.wv.most_similar('кредит')

[('кредит.', 0.8426833748817444),
 ('кредит,', 0.8351993560791016),
 ('ипотека', 0.810340940952301),
 ('автокредит', 0.7542774677276611),
 ('кредита,', 0.7337394952774048),
 ('кредитов.', 0.708098292350769),
 ('кредиту,', 0.6885708570480347),
 ('кредиты,', 0.6867395639419556),
 ('рассрочка', 0.6702771782875061),
 ('досрочно', 0.6552758812904358)]

In [17]:
model.wv.most_similar('депозит')

[('вклад.', 0.9039562940597534),
 ('вклад', 0.8818015456199646),
 ('вклада,', 0.8714506030082703),
 ('вклада.', 0.8640695810317993),
 ('вклады,', 0.8610174059867859),
 ('валютный', 0.859205424785614),
 ('вкладов.', 0.8573611974716187),
 ('вклад,', 0.8537487387657166),
 ('рублёвый', 0.831068754196167),
 ('счета.', 0.8303173780441284)]

In [18]:
model.wv.most_similar(positive=['вклад'], negative=["доллар"])

[('представитель', 0.3753404915332794),
 ('официальный', 0.3739525377750397),
 ('руководство', 0.370221346616745),
 ('читать', 0.3600800931453705),
 ('отзыв', 0.35157185792922974),
 ('сайт', 0.35104408860206604),
 ('действие', 0.3501587510108948),
 ('управлять', 0.34598612785339355),
 ('руководитель', 0.34465643763542175),
 ('разъяснение', 0.3252062201499939)]

In [19]:
model.wv.doesnt_match(["вклад","депозит","коллектор","валюта"])

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'коллектор'

In [20]:
from nltk import FreqDist

In [21]:
top_words = []

fd = FreqDist()
for text in wo_stop:
    fd.update(text)
for i in fd.most_common(100):
    top_words.append(i[0])
print(top_words[:15])

['банка', '-', 'карта', 'весь', 'банк', 'сотрудник', 'деньга', 'кредит', 'сумма', 'отделение', 'дать', 'счёт', 'кредитный', 'день', 'ваш']


In [22]:
top_words = [w for w in top_words if len(w) > 4]
len(top_words)

80

In [23]:
top_words_vec = model.wv[top_words]

In [24]:
tsne = TSNE(n_components=2, random_state=0)

%time top_words_tsne = tsne.fit_transform(top_words_vec)

Wall time: 305 ms


In [34]:
from bokeh.models import ColumnDataSource,LabelSet
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [26]:
def good_bad_coord(word, model):
    gb = np.subtract(model.wv['отлично'], model.wv['плохо'])
    return np.dot(gb,model.wv[word])

In [27]:
def slow_fast_coord(word, model):
    gb = np.subtract(model.wv['быстро'], model.wv['долго'])
    return np.dot(gb,model.wv[word])

In [28]:
bank_names=[]
for resp in responses:
    name = resp['bank_name'].split()[0].lower()
    if (name not in bank_names):
        bank_names.append(name)

In [29]:
x = []
y = []
not_in_list_names = []
for name in bank_names:
    try:
        x.append(good_bad_coord(name,model))
    except:
        not_in_list_names.append(name)
        continue
    try:
        y.append(slow_fast_coord(name,model))
    except:
        print(name)
for n in not_in_list_names:
    bank_names.remove(n) 

In [33]:
output_notebook()
bn = figure(tools="pan,wheel_zoom",
           toolbar_location="above",
           title="X - отлично:плохо, Y - быстро:медленно")

s = ColumnDataSource(data=dict(x=x,
                                    y=y,
                                    names=bank_names))

bn.scatter(x="x", y="y", size=8, source=s)

l = LabelSet(x="x", y="y", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                 source=s, text_align='center')
bn.add_layout(l)

show(bn)

Loading BokehJS ...

In [32]:
import igraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)

ModuleNotFoundError: No module named 'igraph'